In [20]:
import pandas as pd
import numpy as np

# Mapping of station names to their specific file paths
stations_files = {
    "Berlin": {
        "historical": "Berlin Tempelhof produkt_tu_stunde_19510101_20231231_00433.txt",
        "recent": "Berlin Tempelhof produkt_tu_stunde_20230619_20241219_00433.txt"
    },
    "Dresden": {
        "historical": "Dresden-Klotzsche produkt_tu_stunde_19730101_20231231_01048.txt",
        "recent": "Dresden-Klotzsche produkt_tu_stunde_20230619_20241219_01048.txt"
    },
    "Erfurt": {
        "historical": "Erfurt-Weimar produkt_tu_stunde_19510101_20231231_01270.txt",
        "recent": "Erfurt-Weimar produkt_tu_stunde_20230619_20241219_01270.txt"
    },
    "Frankfurt": {
        "historical": "Frankfurt produkt_tu_stunde_19810101_20231231_01420.txt",
        "recent": "Frankfurt Main produkt_tu_stunde_20230619_20241219_01420.txt"
    },
    "Greifswald": {
        "historical": "Greifswald produkt_tu_stunde_19780101_20231231_01757.txt",
        "recent": "Greifswald produkt_tu_stunde_20230619_20241219_01757.txt"
    },
    "Hamburg": {
        "historical": "Hamburg-Fuhlsbüttel produkt_tu_stunde_19490101_20231231_01975.txt",
        "recent": "Hamburg Fühlsbutel produkt_tu_stunde_20230619_20241219_01975.txt"
    },
    "Kiel-Holtenau": {
        "historical": "Kiel Holtenau produkt_tu_stunde_20020101_20231231_02564.txt",
        "recent": "Kiel-Holtenau produkt_tu_stunde_20230619_20241219_02564.txt"
    },
    "Köln": {
        "historical": "Köln:bonn produkt_tu_stunde_19600101_20231231_02667.txt",
        "recent": "Köln:Bonn produkt_tu_stunde_20230619_20241219_02667.txt"
    },
    "München": {
        "historical": "München Flughafen produkt_tu_stunde_19920517_20231231_01262.txt",
        "recent": "München Flughafen produkt_tu_stunde_20230619_20241219_01262.txt"
    },
    "Potsdam": {
        "historical": "Potsdam produkt_tu_stunde_18930101_20231231_03987.txt",
        "recent": "Potsdam produkt_tu_stunde_20230619_20241219_03987.txt"
    },
    "Stuttgart": {
        "historical": "Stuttgart produkt_tu_stunde_19770701_20231231_04928.txt",
        "recent": "Stuttgart produkt_tu_stunde_20230619_20241219_04928.txt"
    }
}

# Function to read and process weather files
def read_weather_file(file_path):
    df = pd.read_csv(
        file_path,
        sep=';',
        parse_dates=['MESS_DATUM'],
        date_format='%Y%m%d%H'
    )
    
    # Drop 'eor' column if present
    if 'eor' in df.columns:
        df.drop(columns='eor', inplace=True)

    # Replace invalid placeholder values (-999.0) with NaN
    df.replace(-999.0, np.nan, inplace=True)

    # Set the datetime column as the index
    df.set_index('MESS_DATUM', inplace=True)

    return df

# Process all stations and store their combined data
stations_data = {}
for station, files in stations_files.items():
    # Read historical and recent files
    df_hist = read_weather_file(files["historical"])
    df_recent = read_weather_file(files["recent"])
    
    # Combine the two DataFrames
    df_combined = pd.concat([df_hist, df_recent]).sort_index()
    
    # Handle duplicates by taking the mean for each timestamp
    df_combined = df_combined.groupby(df_combined.index).mean()
    
    # Store the combined DataFrame
    stations_data[station] = df_combined

# Combine all stations into a single dataset
all_stations = pd.concat(stations_data.values(), axis=1)

# Compute the national average for each metric across all stations
national_average = all_stations.T.groupby(level=0).mean().T

# Keep only the temperature (TT_TU) and humidity (RF_TU) columns
filtered_average = national_average.loc["2022-07-01":"2024-06-30", ['TT_TU', 'RF_TU']]

# Reset the index to make the datetime a regular column
filtered_average.reset_index(inplace=True)

# Rename columns for clarity
filtered_average.rename(columns={'MESS_DATUM': 'DateTime', 'TT_TU': 'AverageTemperature', 'RF_TU': 'AverageHumidity'}, inplace=True)

# Round numeric values to 2 decimal places
filtered_average = filtered_average.round({'AverageTemperature': 2, 'AverageHumidity': 2})

# Save to CSV
filtered_average.to_csv("national_average_temperature_humidity_2022_2024.csv", index=False)

print(filtered_average.head())


             DateTime  AverageTemperature  AverageHumidity
0 2022-07-01 00:00:00               18.73            82.91
1 2022-07-01 01:00:00               17.95            85.82
2 2022-07-01 02:00:00               17.66            87.91
3 2022-07-01 03:00:00               17.17            90.18
4 2022-07-01 04:00:00               17.20            89.73
